In [1]:
import firebase_admin
from firebase_admin import credentials, firestore

# Path to your service account key JSON file
cred = credentials.Certificate('C:/NewDriveD/COding/DecNeuz/DeceNeuz/DeceNeuz_Recommendations/firebase/serviceAccountKey.json')

# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred)

# Initialize Firestore DB
db = firestore.client()

# Reference to a collection
users_ref = db.collection('users')
posts_ref = db.collection('posts')

# Add a new document
# users_ref.add({
#     'name': 'John Doe',
#     'email': 'johndoe@example.com',
#     'age': 30
# })

# Get all documents in a collection
users = users_ref.stream()

for user in users:
    print(f'{user.id} => {user.to_dict()}')

Ct0jOQV7YOeKuPYVcC2yCQETXG52 => {'userImg': 'https://firebasestorage.googleapis.com/v0/b/deceneuz.appspot.com/o/image%2FUser1Pic.png?alt=media&token=78306b6a-d806-4922-ace4-cbc36fa4bf7b', 'created': 1712667354895, 'email': 'user1@gmail.com', 'userId': 'Ct0jOQV7YOeKuPYVcC2yCQETXG52', 'username': 'user1', 'bio': 'Dev#1. The 1st user created on DeceNeuz Platform to Test Features.'}
EhTQqJ9UojeAzKscQOkLhzedacI3 => {'userImg': 'https://lh3.googleusercontent.com/a/ACg8ocJVhNzMkjT77GRRhD-JGlpUnTHT4fER_8qYwHLdOxElBKJjnM8T=s96-c', 'bio': '', 'userId': 'EhTQqJ9UojeAzKscQOkLhzedacI3', 'username': 'Abir Santra', 'email': 'drummer.abir8@gmail.com'}
GbfZhFlrKtP5SlfBSx3FugbfIoG2 => {'userImg': 'https://firebasestorage.googleapis.com/v0/b/deceneuz.appspot.com/o/image%2FUser3Pic.png?alt=media&token=9f64dec3-22c5-4157-baf4-ae84e6927bf1', 'created': 1712671234475, 'email': 'user3@gmail.com', 'userId': 'GbfZhFlrKtP5SlfBSx3FugbfIoG2', 'username': 'user3', 'bio': 'Dev#3. The 3rd user created on DeceNeuz Pla

In [2]:
for post in posts_ref.stream():
    print(f'{post.id} => {post.to_dict()}')

01P5reDt7POvcD9OSegO => {'tags': ['Post'], 'postImg': '', 'userId': 'Ct0jOQV7YOeKuPYVcC2yCQETXG52', 'desc': '<p>Hi, My name is User1 and this is my second post on this platform.</p><p><br></p>', 'pageViews': 16, 'title': '2nd Post from User1', 'created': 1712668314190}
32UxkO1Nmmh2qmi27Smb => {'tags': ['asdf;oiash'], 'postImg': '', 'userId': 'bMhL3XB5HcYItwjssDpE2uXtg6x2', 'desc': "<p>spdo;fikhjsad;'fkj as'd;fj. ads'f;l asd'f;l. fv'; ds;alf k'asd;fl ksad 'f;lk'ads;lfk ';fv '</p>", 'pageViews': 0, 'title': 'baba;lksdgj ;asdlkf ;asldvjads;lfkads;flj ', 'created': 1712827968299}
36TlWc0s5Ye8eOtvqASa => {'tags': ['post'], 'postImg': '', 'userId': 'Ct0jOQV7YOeKuPYVcC2yCQETXG52', 'desc': '<p>Hello</p>', 'pageViews': 2, 'title': 'new post', 'created': 1712693478023}
3BCmAHLuLZT4GuuhOB1W => {'tags': [';alskdjf'], 'postImg': '', 'userId': 'bMhL3XB5HcYItwjssDpE2uXtg6x2', 'desc': '<p>;lasdkjf;alskdjf a;lsdkfj asd;lfkj as;dlkfj as;dlfkj dkls jasd;lfkj ads;flkas ;flkaj s;dflkaj s;dflkajs ;dflkaj ;d

In [4]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [12]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Tokenize descriptions
tokenized_descriptions = [desc.split() for desc in descriptions]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_descriptions, vector_size=100, window=5, min_count=1, workers=4)

# Function to get the vector for a document
def document_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.index_to_key]
    return np.mean(word2vec_model.wv[doc], axis=0)

# Compute document vectors
doc_vectors = np.array([document_vector(desc) for desc in tokenized_descriptions])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(doc_vectors, doc_vectors)

# Function to get similar posts
def get_similar_posts(post_index, similarity_matrix, posts, top_n=5):
    similar_indices = similarity_matrix[post_index].argsort()[:-top_n-1:-1]
    similar_posts = [(posts[i], similarity_matrix[post_index][i]) for i in similar_indices if i != post_index]
    return similar_posts

# Example usage: Get similar posts for the first post
similar_posts = get_similar_posts(0, cosine_sim, posts)
for post, score in similar_posts:
    print(f"Post: {post['desc']}, Similarity Score: {score}")

Post: <p>Hello! This is User1 and this is my First Post.</p>, Similarity Score: 0.7858078479766846
Post: <p>This is a test article</p>, Similarity Score: 0.37807294726371765
Post: <p><span style="color: rgb(150, 224, 114);">Not only did a new study identify two blood proteins linked to cognitive difficulties a year </span></p><p><span style="color: rgb(150, 224, 114);"> after COVID-19 infection, but the What is Lorem Ipsum?</span></p><p><span style="color: rgb(150, 224, 114);"> Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the</span></p><p><span style="color: rgb(150, 224, 114);"> industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type</span></p><p><span style="color: rgb(150, 224, 114);"> and scrambled it to make a type specimen book. It has survived not only five centuries, but also</span></p><p><span style="color: rgb(150, 224, 114);"> the leap into electronic typesetting, remaining essentia

In [5]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Fetch user bio
user_id = 'eNG2Q5DuEmZVIkkIvPXT4PtTkoL2'  # Replace with the actual user ID
user_doc = db.collection('users').document(user_id).get()
print(user_doc.to_dict())
user_bio = user_doc.to_dict().get('bio', '')

# Fetch posts
posts_ref = db.collection('posts')
posts = [post.to_dict() for post in posts_ref.stream()]
print(len(posts))

# Remove words that begin with < and end with > from descriptions
descriptions = [re.sub(r'<[^>]+>', '', post['desc']) for post in posts if 'desc' in post]
sample_post = "Hi, I am a SW engineer. I love animes and sports. I am very fond of listening to Miusician."
descriptions.append(sample_post)
print(descriptions[1])
# Combine user bio and post descriptions
documents = [user_bio] + descriptions

# Tokenize descriptions
tokenized_descriptions = [desc.split() for desc in documents]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_descriptions, vector_size=100, window=5, min_count=1, workers=4)

# Function to get the vector for a document
def document_vector(doc):
    # Remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.wv.index_to_key]
    return np.mean(word2vec_model.wv[doc], axis=0)

# Compute document vectors
doc_vectors = np.array([document_vector(desc) for desc in tokenized_descriptions])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(doc_vectors, doc_vectors)

# Function to get similar posts
def get_similar_posts(post_index, similarity_matrix, posts, top_n=5):
    similar_indices = similarity_matrix[post_index].argsort()[:-top_n-1:-1]
    similar_posts = [(posts[i], similarity_matrix[post_index][i]) for i in similar_indices if i != post_index]
    return similar_posts

# Example usage: Get similar posts for the first post
similar_posts = get_similar_posts(0, cosine_sim, documents)
# for post, score in similar_posts:
#     print(f"Post: {post['desc']}, Similarity Score: {score}")
for post in similar_posts:
    print(f"Post: {post[0]}, Similarity Score: {post[1]}")
# # Compute TF-IDF vectors
# vectorizer = TfidfVectorizer().fit_transform(documents)
# vectors = vectorizer.toarray()

# # Compute cosine similarity between user bio and each post
# cosine_similarities = cosine_similarity(vectors[0:1], vectors[1:]).flatten()

# # Print similarity scores
# for i, score in enumerate(cosine_similarities):
#     print(f'Post: {descriptions[i]}, Similarity Score: {score}')

{'bio': 'Sports freak | Anime lover | Miusician', 'email': 'drccjds0wr@somelora.com', 'created': 1733050993165, 'userId': 'eNG2Q5DuEmZVIkkIvPXT4PtTkoL2', 'username': 'Shinchan Nohara', 'userImg': 'https://firebasestorage.googleapis.com/v0/b/deceneuz.appspot.com/o/image%2Fimages%20(1).jpg?alt=media&token=775fe6f7-b3b1-442c-857a-4aa5713ddfb5'}
56
spdo;fikhjsad;'fkj as'd;fj. ads'f;l asd'f;l. fv'; ds;alf k'asd;fl ksad 'f;lk'ads;lfk ';fv '
Post: I love anime. Big shinchan fan. Chotto misti bou very good musician., Similarity Score: 0.13707707822322845
Post: Not only did a new study identify two blood proteins linked to cognitive difficulties a year  after COVID-19 infection, but the What is Lorem Ipsum? Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but a

In [8]:
import openai
import firebase_admin
from firebase_admin import credentials, firestore
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Initialize Firebase Admin SDK
# cred = credentials.Certificate('path/to/serviceAccountKey.json')
# firebase_admin.initialize_app(cred)
# db = firestore.client()

# Read OpenAI API key from file
with open('C:/NewDriveD/COding/DecNeuz/DeceNeuz/DeceNeuz_Recommendations/DBConnect/openai_api_key.txt', 'r') as file:
    openai.api_key = file.read().strip()

# Fetch user bio
user_id = 'eNG2Q5DuEmZVIkkIvPXT4PtTkoL2'  # Replace with the actual user ID
user_doc = db.collection('users').document(user_id).get()
user_bio = user_doc.to_dict().get('bio', '')

# Fetch posts
posts_ref = db.collection('posts')
posts = [post.to_dict() for post in posts_ref.stream()]

# Extract descriptions
descriptions = [post['desc'] for post in posts if 'desc' in post]

# Function to get text embeddings
def get_embeddings(texts):
    response = openai.Embedding.create(
        input=texts,
        model="text-embedding-ada-002"
    )
    return [embedding['embedding'] for embedding in response['data']]

# Get embeddings for user bio and post descriptions
texts = [user_bio] + descriptions
embeddings = get_embeddings(texts[0])

# Compute cosine similarity between user bio and each post
bio_embedding = np.array(embeddings[0]).reshape(1, -1)
post_embeddings = np.array(embeddings[1:])
cosine_similarities = cosine_similarity(bio_embedding, post_embeddings).flatten()

# Print similarity scores
for i, score in enumerate(cosine_similarities):
    print(f'Post: {descriptions[i]}, Similarity Score: {score}')

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.